In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import pickle
import random
import numpy as np
import sys
import glob
import pandas as pd
from operator import itemgetter, attrgetter

## Word2Vec 기반 데이터 가공하여 train / test 데이터 만들기

- pandas로 csv 파일을 읽는다.
- 각 컬럼은 미리 header 와 dtypes를 정의해둔다.
- 그후 base_model_simple 에 맞게 python list 형태를 생성
    - 단점: python list 생성시 메모리가 너무 많이 필요
    - TODO: base_model_simple을 pandas를 직접 읽어서 처리 (python list 버리자)

In [2]:
D = 100

## 학습 데이터

In [3]:
base_header = ["u", "ts", "v", "slot", "label"]
dtypes = {"u":np.int32, "ts":str, "v":np.int32, "slot":np.int32, "label":np.int32}

In [4]:
name_header = base_header
for i in range(D):
    field = "h{}".format(i)
    name_header.append(field)
    dtypes[field] = np.float32
for i in range(D):
    field = "d{}".format(i)
    name_header.append(field)
    dtypes[field] = np.float32

In [5]:
train_df = pd.read_csv(r'd:\WMIND\temp\train_data_w2vec_1M.csv', header=None, names=name_header, dtype=dtypes, parse_dates=['ts'])
#train_df = pd.read_csv(r'd:\WMIND\temp\train_data_w2vec_100K.csv', header=None, names=name_header, dtype=dtypes, parse_dates=['ts'])

In [6]:
col_dic = dict(zip(list(train_df), range(len(list(train_df)))))

In [7]:
x = train_df[train_df.columns[col_dic['h0']:col_dic['d99']+1]]

In [8]:
y = train_df['label']

In [9]:
oj = y.values

In [10]:
oj.shape

(1549581,)

In [11]:
train_df.columns[col_dic['h0']:col_dic['d99']+1]

Index(['h0', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9',
       ...
       'd90', 'd91', 'd92', 'd93', 'd94', 'd95', 'd96', 'd97', 'd98', 'd99'],
      dtype='object', length=200)

In [12]:
ojm.info()

NameError: name 'ojm' is not defined

In [13]:
train_df = train_df.sample(frac=0.5)

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774790 entries, 1252398 to 1358732
Columns: 205 entries, u to d99
dtypes: datetime64[ns](1), float32(200), int32(4)
memory usage: 614.8 MB


In [15]:
#print(test_df.dtypes)
train_set = []
for row in train_df.itertuples(False):    
    prefix = row[col_dic['u']:col_dic['label']+1]
    profile = row[col_dic['h0']:col_dic['h99']+1]
    deal_feature = row[col_dic['d0']:col_dic['d99']+1]
    train_set.append((prefix, profile, deal_feature))

## 테스트 데이터

In [16]:
test_base_header = ["u", "ts", "p_v", "p_slot", "n_v", "n_slot"]
test_dtypes = {"u":np.int32, "ts":str, "p_v":np.int32, "p_slot":np.int32, "n_v":np.int32, "n_slot":np.int32}

In [17]:
name_header = test_base_header
for i in range(D):
    field = "h{}".format(i)
    name_header.append(field)
    test_dtypes[field] = np.float32
for i in range(D):
    field = "p{}".format(i)
    name_header.append(field)
    test_dtypes[field] = np.float32
for i in range(D):
    field = "n{}".format(i)
    name_header.append(field)
    test_dtypes[field] = np.float32

In [18]:
test_df = pd.read_csv(r'd:\WMIND\temp\test_data_w2vec_1M.csv', header=None, names=name_header, dtype=test_dtypes, parse_dates=['ts'])

In [19]:
###############

In [20]:
col_dic = dict(zip(list(test_df), range(len(list(test_df)))))

In [21]:
pos_columns = list(test_df.columns[col_dic['h0']:col_dic['p99'] + 1])
neg_columns = list(test_df.columns[col_dic['h0']:col_dic['h99'] + 1]) + list(test_df.columns[col_dic['n0']:col_dic['n99'] + 1])

In [22]:
x1 = test_df[pos_columns]
x2 = test_df[neg_columns]

In [23]:
x1 = x1.values
x2 = x2.values

In [24]:
y1 = np.ones((x1.shape[0],))
y2 = np.zeros((x2.shape[0],))

In [25]:
x = np.vstack((x1, x2))
y = np.concatenate((y1, y2))

In [26]:
y

array([1., 1., 1., ..., 0., 0., 0.])

In [62]:
data2 = np.load(r'd:\WMIND\temp\feat\3538943.npy') 

In [63]:
data2.shape

(2048,)

In [ ]:
###############

In [27]:
col_dic = dict(zip(list(test_df), range(len(list(test_df)))))

In [28]:
#print(test_df.dtypes)
test_set = []
for row in test_df.itertuples(False):    
    prefix = row[col_dic['u']:col_dic['n_slot']+1]
    profile = row[col_dic['h0']:col_dic['h99']+1]
    pos_feature = row[col_dic['p0']:col_dic['p99']+1]
    neg_feature = row[col_dic['n0']:col_dic['n99']+1]
    test_set.append((prefix, profile, pos_feature, neg_feature))

### 저장

In [29]:
random.shuffle(train_set)
random.shuffle(test_set)

In [30]:
with open('w2v_dataset.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)